In [1]:
import cv2
import numpy as np
import os
import zipfile

In [2]:
# Path to the zip file containing your labeled dataset
zip_file_path = 'alarms-20231030T094954Z-001.zip'

In [3]:
# Directory where you want to extract the dataset
extraction_dir = 'extraction_directory'

In [4]:
# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

In [5]:
# Load YOLOv2 model
yolov2_cfg = 'yolov2.cfg'
yolov2_weights = 'yolov2.weights'
net = cv2.dnn.readNet(yolov2_weights, yolov2_cfg)

In [6]:
# Load class names
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

In [7]:
# Output video parameters
output_dir = 'output_videos'
os.makedirs(output_dir, exist_ok=True)

In [8]:
# Process each video folder
video_folders = [
    'vehicle_only',
    'spacial_case_with_false_detection',
    'person_vehicle_movement',
    'person_only'
]

In [10]:
for folder in video_folders:
    video_files = [f for f in os.listdir(os.path.join(extraction_dir, folder)) if f.endswith('.mp4')]

    for video_file in video_files:
        video_path = os.path.join(extraction_dir, folder, video_file)
        cap = cv2.VideoCapture(video_path)

        output_file = os.path.join(output_dir, f'{folder}_{video_file}.avi')

        # Used MJPEG codec and match frame rate and resolution
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        out_video = cv2.VideoWriter(output_file, fourcc, 30.0, (frame_width, frame_height))

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # Perform object detection
            blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), [0, 0, 0], swapRB=True, crop=False)
            net.setInput(blob)

            output_layers = net.getUnconnectedOutLayersNames()
            outs = net.forward(output_layers)

            # Process the detected objects
            class_ids = []
            confidences = []
            boxes = []

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]

                    if confidence > 0.5:
                        center_x = int(detection[0] * 640)
                        center_y = int(detection[1] * 480)
                        w = int(detection[2] * 640)
                        h = int(detection[3] * 480)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    confidence = confidences[i]
                    color = (0, 255, 0)  # BGR color code for the bounding box (green)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            out_video.write(frame)

        cap.release()
        out_video.release()

KeyboardInterrupt: 